In [1]:
import requests
import pandas as pd
import time
from sodapy import Socrata
import hashlib

In [2]:
# documentation: https://data.cityofchicago.org/Health-Human-Services/Inpatient-Emergency-Department-and-Outpatient-Visi/7ce8-bpr6/about_data 
client = Socrata("data.cityofchicago.org", app_token="DCuuIo2EqKHeUWr4NKBQKRgaJ", username="kirtanpatel3149@gmail.com", password="Palak_1998$$$$", timeout=120)
results = client.get("7ce8-bpr6", limit=55000)
df_hosp = pd.DataFrame.from_records(results)

In [3]:
df_hosp.shape

(55000, 13)

In [4]:
df_hosp.head(10)


,mmwr_week,week,week_start,week_end,season,data_source,essence_category,respiratory_category,visit_type,demographic_category,demographic_group,percent,current_week_ending
0,202348,48,2023-11-26T00:00:00.000,2023-12-02T00:00:00.000,2023-2024,ESSENCE,CDC COVID-Specific DD v1,COVID-19,ED Visits,Age Group,45_64,0.0219016165,2025-11-29T00:00:00.000
1,202304,4,2023-01-22T00:00:00.000,2023-01-28T00:00:00.000,2022-2023,ESSENCE,CDC Influenza DD v1,Influenza,ED Visits,Age Group,Age Unknown,0,2026-01-31T00:00:00.000
2,201749,49,2017-12-03T00:00:00.000,2017-12-09T00:00:00.000,2017-2018,ESSENCE,CDC Respiratory Syncytial Virus DD v1,RSV,ED Visits,Age Group,05_17,0,2025-12-06T00:00:00.000
3,201845,45,2018-11-04T00:00:00.000,2018-11-10T00:00:00.000,2018-2019,ESSENCE,CDC Broad Acute Respiratory DD v1,Broad Acute Respiratory,ED Visits,Race/Ethnicity,Unknown Race/Ethnicity,0.125376128,2025-11-08T00:00:00.000
4,202323,23,2023-06-04T00:00:00.000,2023-06-10T00:00:00.000,2022-2023,ESSENCE,CDC COVID-Specific DD v1,COVID-19,ED Visits,Age Group,00_04,0.007612457,2026-06-13T00:00:00.000
5,202424,24,2024-06-09T00:00:00.000,2024-06-15T00:00:00.000,2023-2024,ESSENCE,Influenza-like Illness,ILI,ED Visits,Race/Ethnicity,Asian Non-Hispanic,0.008908686,2026-06-20T00:00:00.000
6,202235,35,2022-08-28T00:00:00.000,2022-09-03T00:00:00.000,2021-2022,ESSENCE,CDC COVID-Specific DD v1,COVID-19,Admissions,Age Group,Age Unknown,0,2026-09-05T00:00:00.000
7,202320,20,2023-05-14T00:00:00.000,2023-05-20T00:00:00.000,2022-2023,ESSENCE,CDC COVID-Specific DD v1,COVID-19,Admissions,Age Group,65+,0.016528926,2026-05-23T00:00:00.000
8,201932,32,2019-08-04T00:00:00.000,2019-08-10T00:00:00.000,2018-2019,ESSENCE,CDC Respiratory Syncytial Virus DD v1,RSV,ED Visits,Race/Ethnicity,White Non-Hispanic,0,2026-08-15T00:00:00.000
9,202136,36,2021-09-05T00:00:00.000,2021-09-11T00:00:00.000,2020-2021,ESSENCE,CDC Broad Acute Respiratory DD v1,Broad Acute Respiratory,ED Visits,Age Group,Age Unknown,0,2026-09-12T00:00:00.000


In [5]:
df_hosp.columns

Index(['mmwr_week', 'week', 'week_start', 'week_end', 'season', 'data_source',
       'essence_category', 'respiratory_category', 'visit_type',
       'demographic_category', 'demographic_group', 'percent',
       'current_week_ending'],
      dtype='object')

In [6]:
df_hosp.to_csv('data/hosp_data.csv')

In [7]:
url = "https://aqs.epa.gov/data/api/dailyData/byCounty"

params = {
    "email": "kirtanpatel3149@gmail.com",
    "key": "coppermouse67",
    "param": "88101",      # PM2.5
    "bdate": "20200101",   # start date
    "edate": "20200131",   # end date
    "state": "17",          # Illinois
    "county": "031",         # Cook County
}


In [8]:
dfs = []
for year in range(2015, 2025):
    params["bdate"] = f"{year}0101"
    params["edate"] = f"{year}1231"
    r = requests.get(url, params=params)
    r.raise_for_status()
    d = r.json()
    if "Data" in d:
        dfs.append(pd.DataFrame(d["Data"]))
        print(f"Retrieved {len(d['Data'])} records for {year}.")
    time.sleep(5)
df_daily = pd.concat(dfs)

Retrieved 10274 records for 2015.


Retrieved 10588 records for 2016.


Retrieved 14435 records for 2017.


Retrieved 14740 records for 2018.


Retrieved 16542 records for 2019.


Retrieved 17215 records for 2020.


Retrieved 22596 records for 2021.


Retrieved 31518 records for 2022.


Retrieved 34099 records for 2023.


Retrieved 25894 records for 2024.


In [9]:
df_daily.shape

(197901, 32)

In [10]:
df_daily = df_daily[df_daily['city'] == 'Chicago']
df_daily.shape

(30697, 32)

In [11]:
df_daily.to_csv('data/air.csv')

In [12]:
import os
csv_file_path = "data/air.csv"  
sha_file = "sha256/sha256air_data.txt"
os.makedirs(os.path.dirname(sha_file), exist_ok=True)
with open(csv_file_path, "rb") as f:
    file_bytes = f.read()
hash_value = hashlib.sha256(file_bytes).hexdigest()
with open(sha_file, "w") as f:
    f.write(hash_value)

In [13]:
csv_file_path = "data/hosp_data.csv"  
sha_file = "sha256/sha256hosp_data.txt"
os.makedirs(os.path.dirname(sha_file), exist_ok=True)
with open(csv_file_path, "rb") as f:
    file_bytes = f.read()
hash_value = hashlib.sha256(file_bytes).hexdigest()
with open(sha_file, "w") as f:
    f.write(hash_value)
